In [62]:
from collections import Counter
from dataclasses import dataclass
from pickle import TRUE
from pprint import pprint
from typing import Any, List
from matplotlib import pyplot
import pytorch_lightning as pl
import torch.nn.functional as F
import torch
import albumentations as A

from torch.utils.data import DataLoader2
from torchvision.transforms import transforms
from loguru import logger
from torchvision.datasets import ImageFolder
from torch.cuda.amp.autocast_mode import autocast

import os
import ujson as json
from tqdm import tqdm
import numpy as np
import copy
from pytorch_lightning.loggers import MLFlowLogger
from ClassifierDataset import MultilabelDataset
from TrainClassifierParams import trainParams
from pytorch_lightning.callbacks import ModelCheckpoint

from data import ImportEnv
from loguru import logger as displayLogger
from mlflow.tracking.client import MlflowClient
import itertools
import warnings
import torchvision
import torchmetrics
import dataclasses
import shutil
from torchmetrics.classification.accuracy import MultilabelAccuracy
from torchmetrics import Precision, Recall

from torchmetrics.classification.confusion_matrix import (
    ConfusionMatrix,
    MultilabelConfusionMatrix,
)
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold
import pandas as pd
import torch.nn.functional as F
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import glob
from cleanlab.classification import CleanLearning
import pathlib
from LocalPipeline import getAllParts
import mlflow
from sklearn.metrics import accuracy_score
from skmultilearn.model_selection.measures import get_combination_wise_output_matrix
from skmultilearn.model_selection import (
    iterative_train_test_split,
    label_combination_distribution,
    
)
from skmultilearn.model_selection.iterative_stratification import (
    IterativeStratification
)
from MultilabelKFold import GetDataDir, getAllPart
import plotly.express as px
import plotly.graph_objects as go
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold


In [63]:
def VisDmgLabel(df:pd.DataFrame, allTargetParts:List[str], view:str, kfold : int):
    allDmgParts = []
    for p in allTargetParts:
        dmgCount = df[df[p] == 1][p].count()
        allDmgParts.append(
            {
                "part" : p,
                "dmg_count" : dmgCount,
                "dmg_ratio" : dmgCount / len(df) ,
                'view' : view,
                "kfold" : kfold,
                "model" :f"{view}_{p}"
            }
        )
    return allDmgParts


In [64]:
imgSrcDir, labelSrcDir = GetDataDir()
allParts = getAllPart()
inputDir = labelSrcDir
searchImgView = f"{inputDir}/*.csv"
allSrcAnnFile = glob.glob(searchImgView, recursive=True)
allBalanceMetrics = []
allMultilearnMetrics = []
allGtBalanceMetrics = []
for srcAnnPath in tqdm(allSrcAnnFile, desc="view"):
    imgAngle = srcAnnPath.split("/")[-1].split("_")[1]
    trainParams.srcAnnFile = srcAnnPath
    trainParams.imgAngle = imgAngle
    srcDf = pd.read_csv(trainParams.srcAnnFile)

    allTargetParts = [x for x in srcDf.columns if x in allParts]
    trainParams.runName = imgAngle
    trainParams.targetPart = allTargetParts
    mulitlearnStratify = MultilabelStratifiedKFold(
        n_splits=2,shuffle=True
    )
    skf = KFold(n_splits=2)
    targetPart = trainParams.targetPart
    # srcDf2 = BalanceSampling(srcDf, allTargetParts)
    y = srcDf[allTargetParts]
    X = srcDf["Path"]
    gtLabelDf = VisDmgLabel(y, allTargetParts, view=imgAngle,kfold=0)
    allGtBalanceMetrics.extend(gtLabelDf)
    allSplit = []
    allPreds = []
    for kfoldId, (train_index, test_index) in enumerate(
        skf.split(X, y)
    ):

        # X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y.loc[train_index], y.loc[test_index]
        balanceMetrics = VisDmgLabel(y_train, allTargetParts, view=imgAngle,kfold=kfoldId)
        allBalanceMetrics.extend(balanceMetrics)
    for kfoldId, (train_index, test_index) in enumerate(
        mulitlearnStratify.split(X, y)
    ):

        # X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y.loc[train_index], y.loc[test_index]
        balanceMetrics = VisDmgLabel(y_train, allTargetParts, view=imgAngle,kfold=kfoldId)
        allMultilearnMetrics.extend(balanceMetrics)
kfoldBalanceDf = pd.json_normalize(allBalanceMetrics)
kfoldBalanceDf = kfoldBalanceDf[kfoldBalanceDf["kfold"] == 0]
MultilearnKfoldBalanceDf = pd.json_normalize(allMultilearnMetrics)
MultilearnKfoldBalanceDf = MultilearnKfoldBalanceDf[MultilearnKfoldBalanceDf["kfold"] == 0]
gtKfoldBalanceDf = pd.json_normalize(allGtBalanceMetrics)
gtKfoldBalanceDf = gtKfoldBalanceDf[gtKfoldBalanceDf["kfold"] == 0]



view: 100%|██████████| 6/6 [00:02<00:00,  2.94it/s]


In [65]:
fig = go.Figure()
fig.add_trace(go.Bar(
    x=kfoldBalanceDf["model"], y=kfoldBalanceDf["dmg_ratio"],
    text = kfoldBalanceDf["dmg_ratio"],

    
))
avgVal = kfoldBalanceDf["dmg_ratio"].mean()
fig.add_hline(y=avgVal, line_width=2, line_color="red")
    # print(avgVal)
fig.update_layout(
    yaxis = dict(
        tickmode = 'array',
        tickvals = [np.format_float_positional(avgVal, 2), 0.2, 0.5, 0.7],
    )
)
fig

In [66]:
fig = go.Figure()
fig.add_trace(go.Bar(
    x=MultilearnKfoldBalanceDf["model"], y=MultilearnKfoldBalanceDf["dmg_ratio"],
    text = MultilearnKfoldBalanceDf["dmg_ratio"],

    
))
avgVal = MultilearnKfoldBalanceDf["dmg_ratio"].mean()
fig.add_hline(y=avgVal, line_width=2, line_color="red")
    # print(avgVal)
fig.update_layout(
    yaxis = dict(
        tickmode = 'array',
        tickvals = [np.format_float_positional(avgVal, 2), 0.2, 0.5, 0.7],
    )
)
fig

In [67]:
fig = go.Figure()
fig.add_trace(go.Bar(
    x=gtKfoldBalanceDf["model"], y=gtKfoldBalanceDf["dmg_ratio"],
    text = gtKfoldBalanceDf["dmg_ratio"],

    
))
avgVal = gtKfoldBalanceDf["dmg_ratio"].mean()
fig.add_hline(y=avgVal, line_width=2, line_color="red")
    # print(avgVal)
fig.update_layout(
    yaxis = dict(
        tickmode = 'array',
        tickvals = [np.format_float_positional(avgVal, 2), 0.2, 0.5, 0.7],
    )
)
fig